### preqs
`start bert-en`

In [2]:
import pandas as pd
dfjson = pd.read_json(f'/pi/stack/crawlers/langcrs/all_zh.json')
dfjson.head()

,chapter,index,text,translate,translit,audio
0,The time,1,Excuse me!,对不起 ！,duìbùqǐ!,https://www.book2.nl/book2/ZH/SOUND/0143.mp3
1,The time,2,"What time is it, please?","请问, 现在 几点 了 ?","Qǐngwèn, xiànzài jǐ diǎnle?",https://www.book2.nl/book2/ZH/SOUND/0144.mp3
2,The time,3,Thank you very much.,非常 感谢 ！,Fēicháng gǎnxiè!,https://www.book2.nl/book2/ZH/SOUND/0145.mp3
3,The time,4,It is one o’clock.,现在 一点 。,Xiànzài yīdiǎn.,https://www.book2.nl/book2/ZH/SOUND/0146.mp3
4,The time,5,It is two o’clock.,现在 二点 。,Xiànzài èr diǎn.,https://www.book2.nl/book2/ZH/SOUND/0147.mp3


In [27]:
# quotes['text'].to_list()

In [30]:
dfjson_ja = pd.read_json(f'/pi/stack/crawlers/langcrs/all_ja.json')
dfjson_id = pd.read_json(f'/pi/stack/crawlers/langcrs/all_id.json')
set_ja=set(dfjson_ja['text'].to_list())
set_id=set(dfjson_id['text'].to_list())
print(len(set_ja), len(set_id), set_ja-set_id)

1760 1760 set()


In [31]:
new_df = pd.merge(dfjson_ja, dfjson_id,  how='left', left_on=['text','chapter'], right_on = ['text','chapter'])
new_df.head()

,chapter,index_x,text,translate_x,translit_x,audio_x,index_y,translate_y,translit_y,audio_y
0,People,1,I,私,watashi,https://www.book2.nl/book2/JA/SOUND/0003.mp3,1,saya,,https://www.book2.nl/book2/ID/SOUND/0003.mp3
1,People,2,I and you,私と あなた,watashi to anata,https://www.book2.nl/book2/JA/SOUND/0004.mp3,2,saya dan kamu,,https://www.book2.nl/book2/ID/SOUND/0004.mp3
2,People,3,both of us,私達,watashitachi,https://www.book2.nl/book2/JA/SOUND/0005.mp3,3,kami berdua,,https://www.book2.nl/book2/ID/SOUND/0005.mp3
3,People,4,he,彼,kare,https://www.book2.nl/book2/JA/SOUND/0006.mp3,4,dia (laki-laki),,https://www.book2.nl/book2/ID/SOUND/0006.mp3
4,People,5,he and she,彼と 彼女,kare to kanojo,https://www.book2.nl/book2/JA/SOUND/0007.mp3,5,dia (laki-laki) dan dia (perempuan),,https://www.book2.nl/book2/ID/SOUND/0007.mp3


In [37]:
import json
def search_in(text, lang):
    with open(f'/pi/stack/crawlers/langcrs/all_{lang}.json') as json_file:
        sents=json.load(json_file)
        return [sent for sent in sents if sent['text']==text]
def search_in_list(text, langs):
    rs={}
    for lang in langs:
        rs[lang]=search_in(text, lang)
    return rs

search_in('both of us', 'ja')

[{'chapter': 'People',
  'index': 3,
  'text': 'both of us',
  'translate': '私達',
  'translit': 'watashitachi',
  'audio': 'https://www.book2.nl/book2/JA/SOUND/0005.mp3'}]

In [39]:
search_in_list('I write a letter.', ['ja', 'fa', 'id'])

{'ja': [{'chapter': 'Reading and writing',
   'index': 14,
   'text': 'I write a letter.',
   'translate': '私は 手紙を 書きます 。',
   'translit': 'watashi wa tegami o kakimasu.',
   'audio': 'https://www.book2.nl/book2/JA/SOUND/0116.mp3'}],
 'fa': [{'chapter': 'Reading and writing',
   'index': 14,
   'text': 'I write a letter.',
   'translate': '\u202bمن یک نامه می\u200cنویسم.\u202c',
   'translit': 'man yek nâme râ minevisam.',
   'audio': 'https://www.book2.nl/book2/FA/SOUND/0116.mp3'}],
 'id': [{'chapter': 'Reading and writing',
   'index': 14,
   'text': 'I write a letter.',
   'translate': 'Saya menulis selembar surat.',
   'translit': '',
   'audio': 'https://www.book2.nl/book2/ID/SOUND/0116.mp3'}]}

In [4]:
import pandas as pd
from bert_serving.client import BertClient

# Bert Client must be running locally
bc = BertClient()

In [5]:
def train(quotes):
    embeddings = bc.encode(quotes.text.to_list())
    quotes['EMBEDDINGS'] = embeddings.tolist()

    # Persist to pickle
    quotes.to_pickle('data/embedded_corpus.pkl')
train(dfjson)

In [6]:
def load_quotes_and_embeddings(file):
    quotes = pd.read_pickle(file)

    # change dtype in place for memory efficiency
    quotes['EMBEDDINGS'] = quotes['EMBEDDINGS'].apply(
        lambda arr: np.array(arr, dtype='float32')
    )

    quote_embeddings = np.stack(quotes.EMBEDDINGS.values)

    # reduce memory footprint by dropping column
    quotes.drop('EMBEDDINGS', axis='columns')

    # normalize embeddings for cosine distance
    embedding_sums = quote_embeddings.sum(axis=1)
    normed_embeddings = quote_embeddings / embedding_sums[:, np.newaxis]
    return quotes, normed_embeddings

quotes, embeddings = load_quotes_and_embeddings('data/embedded_corpus.pkl')

In [8]:
def create_index(embeddings):
    import faiss
    """
    Create an index over the quote embeddings for fast similarity search.
    """
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

index = create_index(embeddings)

In [19]:
def search(text):
    text_embedding = bc.encode([text])
    normalized_text_embedding = text_embedding / text_embedding.sum()

    _, idx = index.search(normalized_text_embedding, 5)

    relevant_quotes = quotes.iloc[idx.flatten()]['text'].values
    relevant_chapters = quotes.iloc[idx.flatten()]['chapter'].values
    return relevant_quotes, relevant_chapters

In [20]:
# bc = BertClient()
text="I dreamed a dream."

relevant_quotes, relevant_chapters=search(text)
for q in range(5):
    print('>'+relevant_quotes[q])
    print(relevant_chapters[q])

>I read a letter.
Reading and writing
>I read a book.
Reading and writing
>I want to sleep.
to need – to want to
>I read a sentence.
Reading and writing
>He wrote a letter.
Past tense 1


In [13]:
text="I write a letter."

relevant_quotes, relevant_chapters=search(text)
for q in range(5):
    print('>'+relevant_quotes[q])
    print(relevant_chapters[q])

>I write a letter.
Reading and writing
>I write a book.
Reading and writing
>I write.
Reading and writing
>I write.
Reading and writing
>I write a sentence.
Reading and writing


In [22]:
x,y=[1,2]
print(y)

2
